In [2]:
import sys
sys.path.append("D:/Yuichi/MemoryAge_Transformer_xLARGE_using_FREEZING_ONLY")
# the code needs to search for the modules in the parent directory

import logging
import os
import random
import time
import os
import multiprocessing
CPU_num = multiprocessing.cpu_count()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch.utils.data.dataloader import DataLoader
from torcheeg import transforms
from torcheeg.datasets import MemoryAgeDataset
from torcheeg.model_selection import KFoldCrossSubject
from torcheeg.models import LFP_Transformer
from torcheeg.trainers import ClassifierTrainer

#### Set the random number seed in all modules to guarantee the same result when running again.

In [3]:
def seed_everything(seed):
    
    # Sets the random seed for Python's built-in random library. This will affect random operations like random.shuffle or random.randint.
    random.seed(seed) 

    #Sets the random seed for NumPy. This will affect random operations in NumPy like np.random.randn or np.random.shuffle.
    np.random.seed(seed)

    # Sets the random seed for hash functions in Python. The hash functions are used in Python dictionaries and sets,
    # and this ensures that they behave the same way each time you run the program.
    os.environ["PYTHONHASHSEED"] = str(seed)

    # Sets the random seed for PyTorch's CPU random number generator. This will affect random operations in PyTorch like random tensor generation.
    torch.manual_seed(seed)

    # Sets the random seed for PyTorch's CUDA random number generator, which affects random number generation on the GPU.
    torch.cuda.manual_seed(seed)

    # Ensures that every time you run your code, the algorithms used by cuDNN produce deterministic results.
    # NOTE: By default, cuDNN uses algorithms that are optimized for performance, but these algorithms might use approximations or other shortcuts 
    # that can result in small differences in output. This is known as being "non-deterministic."
    torch.backends.cudnn.deterministic = True  

    # Disables the cudNN auto-tuner, which optimizes cuDNN to find the best algorithm to use for your hardware. 
    # By disabling this, you make sure that cuDNN uses a deterministic algorithm.
    # NOTE: The cuDNN library provides several algorithms to perform operations commonly used in deep learning, like convolutions and pooling.
    # These algorithms can vary in their internal implementations and may have different performance and memory characteristics. 
    # The cuDNN auto-tuner is designed to automatically select the most efficient algorithm for your specific hardware configuration. 
    torch.backends.cudnn.benchmark = False

random_state = 19890516
seed_everything(random_state)

In [4]:
def load_val_batches(val_loader, num_batches=5):
    '''
    This function is designed to load a small number of batches from the validation set.
    I will load validation set in different runs, and compare whether the samples are in the same order.
    '''
    batches = []
    for i, batch in enumerate(val_loader):
        if i >= num_batches:
            break
        batches.append(batch)
    return batches

# Building Deep Learning Pipelines Using TorchEEG

#### Step 1: Initialize the Dataset

In [5]:
dataset = MemoryAgeDataset(io_path='./processed_dataset/',  
                           root_path='./raw_dataset/data_memory_age',
                           online_transform=transforms.ToTensor(),     # used in __getitem__()
                           num_worker=1,
                           io_mode='pickle')

dataset does not exist at path ./processed_dataset/, generating files to path...


[PROCESS]: 100%|██████████| 24/24 [02:12<00:00,  5.53s/it]

dataset already exists at path ./processed_dataset/, reading from path...


In [6]:
dataset.info

,subject_id,clip_id,label,trial_id
0,ym212,ym212_Recent_0,0,0
1,ym212,ym212_Recent_1,0,0
2,ym212,ym212_Recent_2,0,0
3,ym212,ym212_Recent_3,0,0
4,ym212,ym212_Recent_4,0,0
...,...,...,...,...
18611,ym227,ym227_Remote_500,1,1
18612,ym227,ym227_Remote_501,1,1
18613,ym227,ym227_Remote_502,1,1
18614,ym227,ym227_Remote_503,1,1


#### Step 2: Divide the Training and Test samples in the Dataset

In [7]:
k_fold = KFoldCrossSubject(n_splits=12, shuffle=False, split_path='./processed_dataset/cross_subject_k_fold_spilt')
# shulffle=False, so the order of the subjects in test is ym212, ym213, ..., ym227
# n_spilts=the num of subjects, so leave-one-subject-out cross validation

#### Step 3: Define the Model and Start Training

In [8]:
for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    
    # ------------------------------------------- 
    # Initialize the model
    model = LFP_Transformer(depth=3)  # depth=3, so the model has 3 transformer blocks, xLARGE model
    
    # ------------------------------------------- 
    # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
    # The major parameters of the trainer are defined in ClassifierTrainer
    trainer = ClassifierTrainer(model=model,
                                lr=1e-4,
                                devices=1,
                                accelerator='gpu',
                                num_classes=2,
                                weight_decay=1e-4)

    # ------------------------------------------- 
    # Initialize several batches of training samples and test samples
    # Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset.
    train_loader = DataLoader(train_dataset,
                              batch_size=128,
                              shuffle=True,
                              num_workers=CPU_num)
    
    val_loader = DataLoader(val_dataset,
                            batch_size=128,
                            shuffle=False,
                            num_workers=CPU_num)
    # shuffle=False, very important, do not shuffle the samples in the validation set
    # in the validation and test phase, just feed all the samples in the default order,
    # for example, in fold0, the order is ym212_Recent_0, ym212_Recent_1, ym212_Recent_2, ...

    # ------------------------------------------- 
    # Do 50 rounds of training
    trainer.fit(train_loader, val_loader, fold_idx=i, max_epochs=30)

    # ------------------------------------------- 
    # Test the model on the test set
    # during the test period, for each batch with batch_size samples, the desired attention scores, 
    # i.e., the attention scores between the cls token and the patch tokens in the first transformer block,
    # are saved in the trainer.saved_attention, which is a list with several arrays (shape = (batch_size, num_heads, num_patches))
    trainer.test(val_loader, fold_idx=i)

    # ----- Process the predictions of the test samples
    saved_predictions = trainer.saved_predictions_all
    saved_predictions = np.concatenate(saved_predictions, axis=0)  # shape = (num_samples, num_classes) = (num_samples, 2)
    # apply softmax to each row, namely, the predictions for each sample
    saved_predictions = np.exp(saved_predictions) / np.sum(np.exp(saved_predictions), axis=1, keepdims=True)
    # to pandas dataframe. the first column is 'class 0 probability', the second column is 'class 1 probability'
    saved_predictions = pd.DataFrame(saved_predictions, columns=['class 0 probability', 'class 1 probability'])
    # load the corresponding test samples
    test_sample_info = pd.read_csv(f'./processed_dataset/cross_subject_k_fold_spilt/test_fold_{i}.csv')
    # concatenate the test sample info and the predictions
    test_sample_info = pd.concat([test_sample_info, saved_predictions], axis=1)
    # save the test sample info and the predictions
    test_sample_info.to_csv(f'./processed_dataset/cross_subject_k_fold_spilt/test_fold_{i}.csv', index=False)

    # ----- Process the attention scores
    # Access the saved attention matrices in the TEST phase
    attention_matrices = trainer.saved_attention_all
    # concat attention_matrices in the batch dimension
    attention_matrices = np.concatenate(attention_matrices, axis=0)  # shape = (num_samples, num_heads, num_patches) = (num_samples, 4, 17)
    # create a folder in ./processed_dataset/ to save the attention matrices if not exist
    if not os.path.exists('./processed_dataset/attention_scores/'):
        os.makedirs('./processed_dataset/attention_scores/')
    np.save('./processed_dataset/attention_scores/attention_from_the_test_subject_in_Fold%d.npy' % i, attention_matrices)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


-------------------- FOLD 0 -------------------
train_subjects: ['ym227', 'ym213', 'ym219', 'ym223', 'ym226', 'ym222', 'ym224', 'ym220', 'ym218', 'ym214', 'ym215']
test_subjects: ['ym212']
------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | LFP_Transformer  | 546 K 
1 | _ce_fn        | CrossEntropyLoss | 0     
2 | train_loss    | MeanMetric       | 0     
3 | val_loss      | MeanMetric       | 0     
4 | test_loss     | MeanMetric       | 0     
5 | train_metrics | MetricCollection | 0     
6 | val_metrics   | MetricCollection | 0     
7 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
546 K     Trainable params
0         Non-trainable params
546 K     Total params
2.184     Total estimated model params size (MB)



Epoch 0: [Val] val_loss: 1.253  val_f1score: 0.000  val_accuracy: 0.000  

Metric val_loss improved. New best score: 0.629


...


Epoch 0: [Val] val_loss: 0.701  val_f1score: 0.507  val_accuracy: 0.562     [Train] train_loss: 0.674  train_f1score: 0.532  train_accuracy: 0.579  
Epoch 1: [Val] val_loss: 0.726  val_f1score: 0.543  val_accuracy: 0.549     [Train] train_loss: 0.601  train_f1score: 0.665  train_accuracy: 0.673  
Epoch 2: [Val] val_loss: 0.752  val_f1score: 0.535  val_accuracy: 0.609     [Train] train_loss: 0.541  train_f1score: 0.722  train_accuracy: 0.726  
Epoch 3: [Val] val_loss: 0.798  val_f1score: 0.550  val_accuracy: 0.627     [Train] train_loss: 0.491  train_f1score: 0.758  train_accuracy: 0.761  
Epoch 4: [Val] val_loss: 0.735  val_f1score: 0.598  val_accuracy: 0.634     [Train] train_loss: 0.434  train_f1score: 0.794  train_accuracy: 0.797  


Monitored metric val_loss did not improve in the last 5 records. Best score: 0.701. Signaling Trainer to stop.


Epoch 5: [Val] val_loss: 0.766  val_f1score: 0.622  val_accuracy: 0.652     [Train] train_loss: 0.381  train_f1score: 0.829  train_accuracy: 0.830  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ./processed_dataset/Model_Checkpoints/Best_model_Fold11.ckpt


[Test] test_loss: 0.701 test_f1score: 0.507 test_accuracy: 0.562 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.5622119903564453
      test_f1score           0.507488489151001
        test_loss            0.701109766960144
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
